In [1]:
from utils.db_get_utils import get_filtered_gate_pass_times
import pandas as pd
from datetime import datetime, time, timedelta
from IPython.display import display, HTML
import shutil
pd.set_option('display.width', 1000) 
pd.set_option('display.max_columns', None)

In [2]:
yanuar_pass_list = get_filtered_gate_pass_times()
df = pd.DataFrame(yanuar_pass_list)
print(df)


df['timestamp'] = pd.to_datetime(df['timestamp'])
print(df.head(10))

      id  isHoliday         timestamp pass_type  pass_type_id  person_id  gate_id
0    674       True  2024-01-02 00:04      ВХОД             1      78029       56
1    410       True  2024-01-02 02:56      ВХОД             1      39085       10
2      1       True  2024-01-02 07:25      ВХОД             1      91048       20
3      2       True  2024-01-02 07:26      ВХОД             1      91048       56
4      3       True  2024-01-02 10:25     ВЫХОД             2      91048       20
..   ...        ...               ...       ...           ...        ...      ...
502  557      False  2024-12-30 14:53     ВЫХОД             2      42250       20
503  467      False  2024-12-30 18:04     ВЫХОД             2      39085       56
504  645      False  2024-12-31 00:17     ВЫХОД             2      78029       20
505  870      False  2024-12-31 02:32      ВХОД             1      20312       56
506  871      False  2024-12-31 21:19     ВЫХОД             2      20312       56

[507 rows x 7 c

In [3]:
end_date = '2024-01-28'
df = df[(df['person_id'] == 91048) & (df['timestamp'] < end_date) ]
df = df.sort_values(by=['person_id', 'timestamp'])
print(df)

    id  isHoliday           timestamp pass_type  pass_type_id  person_id  gate_id
2    1       True 2024-01-02 07:25:00      ВХОД             1      91048       20
3    2       True 2024-01-02 07:26:00      ВХОД             1      91048       56
4    3       True 2024-01-02 10:25:00     ВЫХОД             2      91048       20
5    4       True 2024-01-02 11:30:00      ВХОД             1      91048       20
6    5       True 2024-01-02 17:25:00     ВЫХОД             2      91048       56
19  76      False 2024-01-09 02:37:00      ВХОД             1      91048       10
20  77      False 2024-01-09 18:00:00     ВЫХОД             2      91048       10
22  54      False 2024-01-10 08:02:00      ВХОД             1      91048       20
25  55      False 2024-01-10 17:02:00     ВЫХОД             2      91048       20
27  46      False 2024-01-11 02:50:00      ВХОД             1      91048       10
28  47      False 2024-01-11 21:25:00     ВЫХОД             2      91048       10
33  74       Tru

In [4]:
#df['group_id'] = (df['pass_type'] != df['pass_type'].shift()).cumsum()
df['date'] = df['timestamp'].dt.date 
work_times = []
# Групируем по дням , что бы обойти все дни
for person_id, group in df.groupby('person_id'):
    # Добовляем новый столбец в группу который будет обозначать уникальный номер последовательности входов или выходов
    # Если идет подряд записи о входе (или выходе) то в столбец попадет одинаковое число
    group['group_id'] = (group['pass_type'] != group['pass_type'].shift()).cumsum()
    # Группируем по данному столбцу
    start_data = None
    for _, group_id in group.groupby('group_id'):
        type_pass_group = group_id.iloc[0]['pass_type']
        if type_pass_group == 'ВХОД':
            start_data = group_id.iloc[0]['timestamp']
        else:
            #если был выход считаем дельту и добавляем запись в массив , дату ставим 
            if start_data is not None :
                #берем последнее время выхода
                end_data = group_id.iloc[-1]['timestamp']
                #считаем дельту , если дельта меньше чем 18 часов , то считаем что данные верны
                work_duration = (end_data - start_data).total_seconds() / 3600
                if work_duration <= 18:
                    work_times.append({
                        'person_id':person_id,
                        'start_time': start_data,
                        'end_data':end_data,
                        'date':start_data.date(),
                        'work_hours':work_duration
                    })
                else:
                # обнуляем начальный вход и считаем что данные не действительны
                    print(f"Недействительные данные сотрудник id={person_id} зашел в {start_data} и вышел в {end_data}. что в сумме больше 18 часов")
                start_data = None

result_df = pd.DataFrame(work_times)
print(result_df)

        
    

Недействительные данные сотрудник id=91048 зашел в 2024-01-11 02:50:00 и вышел в 2024-01-11 21:25:00. что в сумме больше 18 часов
Недействительные данные сотрудник id=91048 зашел в 2024-01-14 01:20:00 и вышел в 2024-01-15 00:53:00. что в сумме больше 18 часов
   person_id          start_time            end_data        date  work_hours
0      91048 2024-01-02 07:25:00 2024-01-02 10:25:00  2024-01-02    3.000000
1      91048 2024-01-02 11:30:00 2024-01-02 17:25:00  2024-01-02    5.916667
2      91048 2024-01-09 02:37:00 2024-01-09 18:00:00  2024-01-09   15.383333
3      91048 2024-01-10 08:02:00 2024-01-10 17:02:00  2024-01-10    9.000000
4      91048 2024-01-25 04:59:00 2024-01-25 20:32:00  2024-01-25   15.550000
